In [1]:
#packages

In [2]:
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup

from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

stm = PorterStemmer()
wnl = WordNetLemmatizer()

In [3]:
# reading files

In [4]:
links = pd.read_csv('PostLinks.csv')
posts = pd.read_csv('Posts.csv')

In [5]:
# Graph Modelling

In [6]:
# vertices list

In [7]:
%%time

u_source = links['PostId'].unique().tolist()
u_target = links['RelatedPostId'].unique().tolist()

u_source.extend(u_target)

Vertices_list = list(dict.fromkeys(u_source))
Vertices_list.sort()
len(Vertices_list)

Wall time: 1.97 ms


2245

In [8]:
#edge_list 

In [9]:
edge_list = []
Edges_list = list(zip(links['PostId'],links['RelatedPostId']))
Edges_list.sort()
Edges_unique = list(dict.fromkeys(Edges_list))
len(Edges_unique)

1558

In [10]:
# Adjacency list

In [11]:
%%time
V = len(Vertices_list)

Adj_list = []

for s in range(V):
    Adj_list.append([])

for i in Edges_unique:
    a = Vertices_list.index(i[0])
    Adj_list[a].append(i[1])

Wall time: 34.9 ms


In [12]:
#segregating src and adjacent target nodes

In [13]:
src_nodes = [] 
target_nodes = []

for i in Adj_list:
    if len(i)>1:
        ind = Adj_list.index(i)
        src_nodes.append(Vertices_list[ind])
        target_nodes.append(i)
        
src_nodes_u = list(dict.fromkeys(src_nodes))

# Hard coding to remove duplicates from target_nodes

p = [66,73,74,94,119,120]
for i in p:
    target_nodes.pop(i)

In [14]:
#### RANKING RELATED POSTS - hyp(1)

In [15]:
src_posts = posts[posts.Id.isin(src_nodes)]
k = [i for i in src_nodes if i not in list(src_posts['Id']) ]
try:
    for i in k:
        target_nodes.pop(src_nodes_u.index(i))
        src_nodes_u.pop(src_nodes_u.index(i))
except:
    print('no element fround')
        

In [16]:
src_scrapped = [] 

for post in src_posts['Title']:
    soup = BeautifulSoup(post, 'html.parser')

    article = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', soup.text, flags=re.MULTILINE)

    article = article.replace('\n','')

    src_scrapped.append(article)
    

    # with open('posts.txt', 'w') as file:
    #     file.write(article)


In [17]:
# list to be populated in dropdown
# src_scrapped 

In [18]:
target_nodes[:1],src_nodes_u[:1]

([[739, 808]], [1141])

In [19]:
# action on submit

In [20]:

def relAnswer(question):
    
    trg_scrapped = []
    
    sel_que = question
    
    q_ind = src_scrapped.index(sel_que)

    # adjacent nodes of selected question 
    
    answers_ind = target_nodes[q_ind]
    
    soup_q = BeautifulSoup(sel_que, 'html.parser')    
    
#     scrapped_que = ''

    scrapped_que = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', soup_q.text, flags=re.MULTILINE)

    scrapped_que = scrapped_que.replace('\n','')
    
    
    for ind in answers_ind:
        
        ans = posts['Title'][posts['Id']==ind]

        ans = ans.tolist()

        if len(ans)>0:

            soup = BeautifulSoup(ans[0], 'html.parser')

            article = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', soup.text, flags=re.MULTILINE)

            article = article.replace('\n','')

            trg_scrapped.append(article)
        
    return trg_scrapped,scrapped_que,q_ind # all answers along with the selected question

    

In [21]:
similarity_measure = []
post_col = []
sim_col = []
relpost_col = []

def onsubmit(Post):

    Answers,selected_que,q_ind = relAnswer(Post) #uncomment when function is defined
# Answers,selected_que = relAnswer('Some suggestion for career in data science or predictive modeling')

    pre_p_ans = []
    

    stop_words = stopwords.words('english')
    # print(stop_words)
    l_w_ans = []
    list_words_sw = []
    for answer in Answers:
        pun = ['!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '_', '+', '=', '{', '}', '[', ']', '|', '\\', ":", ";", '"', '<', ',', '>', '.', '?', '/']
        for i in pun : 
            answer = answer.replace(i,' ')

        answer = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', answer, flags=re.MULTILINE)
    #     answer = re.sub(r'[a-z]*[A-Z]*[:.]+\S+','',answer,flags=re.MULTILINE)
        answer =answer.lower()
        answer =answer.replace('i\'m','i am')
        answer =answer.replace("'",' ')
        answer =answer.translate(str.maketrans(' ', ' ', string.punctuation))

        pre_p_ans.append(answer)
        list_words = TreebankWordTokenizer().tokenize(answer)

        for word in list_words:
            tmp = wnl.lemmatize(word)
            tmp = stm.stem(word)
            l_w_ans.append(tmp)

        list_words_sw.append([words for words in l_w_ans if words not in stop_words])

    # TF_IDF - similarity measure for posts

    pre_p_ans.append(selected_que.lower())
    vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
    tfidf = vect.fit_transform(pre_p_ans)                                                                                                                                                                                                                       
    pairwise_similarity = tfidf * tfidf.T 
    sim_posts_array = pairwise_similarity.toarray()
    for row in range(len(sim_posts_array)) :
        for element in range(len(sim_posts_array[row])) :
            if row == element:
                     sim_posts_array[row][element] = 0

    sim_posts_ind = sim_posts_array[-1]


    similarity_measure.append(sim_posts_ind[:-1].tolist())

#     print(similarity_measure)

#     print(pairwise_similarity)
#     print(sim_posts_ind.argmax())

    # 3-columns as list for post, sim, relatedposts
#     print(pre_p_ans)

    for i in range(len(pre_p_ans)-1):
        post_col.append(pre_p_ans[-1])
        relpost_col.append(pre_p_ans[i])    
        sim_col.append(similarity_measure[q_ind][i])
        

    return_string = pre_p_ans[sim_posts_ind.argmax()]

    return return_string   #return the answer which is most similar to selected question


In [23]:
relPost_list = []
for post in src_scrapped:
    relPost_list.append(onsubmit(post))
    

In [24]:
#csv of post with most similar related post

In [25]:
relPost_df = pd.DataFrame()

In [26]:
relPost_df['Posts'] = src_scrapped
relPost_df['RelatedPost'] = relPost_list

In [27]:
relPost_df.to_csv('SimilarPosts.csv', index=False)

In [31]:
sim_post_ids = pd.DataFrame()
sim_post_ids['postid'] = src_nodes_u
sim_post_ids['postid_rp'] = target_nodes
sim_post_ids['most_sim'] = 
sim_post_ids.to_csv('SimPost_ids.csv',index=False)

In [32]:
#csv triple store format (post- similarity - related post)

In [33]:
tsf_post = pd.DataFrame()
tsf_post['Post'] = post_col
tsf_post['Similarity Measure'] = sim_col
tsf_post['Related Posts'] = relpost_col
tsf_post.to_csv('SimilarityMeasure_TripleStore.csv',index=False)

In [34]:
#knowledge graph out of tsf_post

In [35]:
from py2neo import Graph, Relationship, Node

In [36]:
g = Graph(uri="http://localhost:7474",user="neo4j",password="1234")
g.delete_all
kg = g.begin()

In [37]:
%%time
for ind in range(len(tsf_post)):
    
    p = Node('Post', title = tsf_post['Post'][ind])
    kg.merge(p,'Post', 'title')
    
    rp = Node('Post',title = tsf_post['Related Posts'][ind])
    kg.merge(rp, 'Post','title')
    
    sim = Relationship(p , str(tsf_post['Similarity Measure'][ind]) , rp)
    kg.create(sim)
    
kg.commit()

Wall time: 14.5 s
